In [2]:
!pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 665.0 kB/s eta 0:00:001m754.9 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 930.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Created wheel for peewee: filename=peewee-3.18.2-cp312-cp312-macosx_11_0_arm64.whl size=272211 sha256=83027b0323dd8b8bf6dc8510b510d9c98a5940af258223e87119503f95ede9d4
  Stored in directory: /Users/anshulkedia/Library/Caches/pip/wheels/d1/df/a9/0202b051c65b11c992dd6db9f2babdd2c44ec7d35d511be5d3
Successfully built peewee


In [3]:
# Download stock data
TICKER = 'RELIANCE.NS'
data = yf.download(TICKER, start='2015-01-01', end='2024-01-01')
close_prices = data[['Close']]

/var/folders/d9/qpj9c7z11zl2_410n7td44w40000gn/T/ipykernel_4850/4011050199.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(TICKER, start='2015-01-01', end='2024-01-01')
[*********************100%***********************]  1 of 1 completed


In [4]:
# Preprocessing
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(close_prices)

In [5]:
X, y = [], []
n_input = 60
n_output = 7
for i in range(n_input, len(scaled_data) - n_output):
    X.append(scaled_data[i - n_input:i])
    y.append(scaled_data[i:i + n_output])

X, y = np.array(X), np.array(y)

In [6]:
# Build LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(100))
model.add(Dense(n_output))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=20, batch_size=32)

Epoch 1/20


/Users/anshulkedia/miniconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0731
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0015
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0013
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0012
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0013
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0012
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0012
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.5476e-04
Epoch 9/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0012
Epoch 10/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0013
Epoch 11/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.9081e-04
Epoch 12/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.2503e-04
Epoch 13/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 8.7108e-04
Epoch 14/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 9.3124e-04
Epoch 15/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 8.

In [8]:
# Save model and scaler
model.save('lstm_7day_model.keras')
joblib.dump(scaler, 'scaler.save')

['scaler.save']